In [2]:
import fitz  # PyMuPDF
from PIL import Image
import os
from tqdm import tqdm 

In [3]:
def get_filename_without_extension(file_path):
    return os.path.splitext(os.path.basename(file_path))[0]

In [4]:
def pdf_to_images(pdf_path, output_folder):
    # PDFファイルを開く
    pdf_document = fitz.open(pdf_path)
    file_name    = get_filename_without_extension(pdf_path)

    # 各ページを処理
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        
        # ページを画像としてレンダリング
        pix = page.get_pixmap()
        
        # 画像をPillow Imageオブジェクトに変換
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # 画像を保存
        img.save(f"{output_folder}/{file_name}_{page_num + 1}.png")


In [5]:
#フォルダパス内のファイルパスを取得する関数
def get_file_paths(folder_path):
    file_paths = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

In [18]:
#set_path
input_folder_path = "./input/"
output_folder_path = "./input/pdf_img"

pdf_folder_path = input_folder_path +"pdf/"
pdf_text_folder_path = input_folder_path +"pdf_text/"

### 1.transform pdf to img

In [7]:
# get_path
pdf_paths = get_file_paths(pdf_folder_path)
# pdf_paths


In [8]:
# #transfer pdf to img
# for pdf_path in tqdm(pdf_paths):
#     pdf_to_images(pdf_path, output_folder_path)

### 2.summarize text in img

In [9]:
import os
import base64
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [10]:

from multiprocessing import Pool, cpu_count


In [11]:
#set env
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
API_VERSION = os.getenv("API_VERSION")
DEPLOYMENT_ID_FOR_CHAT_COMPLETION = os.getenv("DEPLOYMENT_ID_FOR_CHAT_COMPLETION")

In [12]:
def get_text_from_img(image_file):
    with open(image_file, 'rb') as f:
        img = f.read()
        content =  [
        {
            'role' : 'system',
            'content' : [
                {'type':'text', 'text':'あなたは画像から文字を読み取りmarkdownに変換して出力します'},
                {'type':'text', 'text':'わからないときは # unknown と出力します'},
                ]
        },
        {
            'role' : 'user',
            'content' : [
            { 'type': "image_url", 'image_url': {'url':'data:image/jpeg;base64,'+ base64.b64encode(img).decode('ascii') }},
            ]
        }
        ]

    client = AzureOpenAI(
        api_key=AZURE_OPENAI_API_KEY,
        api_version=API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT)

    completion_resp = client.chat.completions.create(
        model=DEPLOYMENT_ID_FOR_CHAT_COMPLETION,
        temperature=0,
        seed=42,
        messages=content
        )
    
    return completion_resp.choices[0].message.content

In [13]:
# get_path
img_paths = get_file_paths(output_folder_path)


In [19]:
#abstract_text_and_save
for img_path in tqdm(img_paths):
    md_text   = get_text_from_img(img_path)
    file_name = get_filename_without_extension(img_path)
    with open( pdf_text_folder_path+ f'{file_name}.md', 'w') as f:
        f.write(md_text)
        f.close()


    print("====================")
    print(pdf_text_folder_path+ f'{file_name}.md')
    print(md_text)
    print("====================")


  0%|          | 1/1539 [00:06<2:35:48,  6.08s/it]

./input/pdf_text/10_1.md
```markdown
# Sustainability Data Book 2024

パナソニックグループ  
サステナビリティ データブック 2024

**Panasonic Group**
```


  0%|          | 2/1539 [00:15<3:23:00,  7.92s/it]

./input/pdf_text/10_10.md
# 環境

## 方針
当社グループは創業以来、環境社会に貢献することを経営理念として、1970年代から環境保全に取り組んできました。そして、1991年、環境経営を基本方針（環境基本方針）に位置付け、1993年には環境管理システムを構築し、環境保全活動を推進してきました。安全な水と地球環境の保全は、私たちの事業活動において重要なテーマです。

2001年に策定した「グリーンプラン2018」の活動計画、2010年には「グリーンプラン2018」を改訂し、2018年には新たに「グリーンプラン2020」を策定しました。水、エネルギー、廃棄物などの環境負荷を低減するため、活動を推進しています。

2013年、当社グループは「人と社会にとってより良い世界」を実現するために「グリーンプラン2018」を策定し、B2B環境経営の取り組みを強化することとしました。同年、環境省が主催する「国際連携のための環境経営システム」において、クリーンエネルギーの利用促進に関する活動を目指すことを表明しました。さらに、2050年には、温室効果ガスの排出量を実質ゼロにすることを目指しています。

2019年、グリーンプラン2018の成果を受け、パナソニックグループの創立・運営に関する方針を見直し、環境マネジメントシステムを強化しました。


  0%|          | 3/1539 [00:24<3:35:35,  8.42s/it]

...

./input/pdf_text/10_108.md
```markdown
# 責任ある調達活動

## 方針
- 責任者・体制
- サプライチェーンのデュー・ディリジェンス
- 環境負荷低減の取り組み
- 責任ある調達活動
- 社内教育と社外での啓発活動
- 通報・相談窓口

## 2022年12月20日 CSR ガイドライン改訂時からの基準を満たしています。

### CSR自主アセスメントの実施と監査
当社グループは、サプライチェーンにおける人権・環境・ディリジェンスに関してCSR自主アセスメントを実施しています。結果を踏まえ、CSR自主アセスメントは、当グループのCSRガイドラインに基づき実施されており、毎年の実施結果をもとに、改善の必要性を検討しています。新規の取引先に対しても、同様に実施され、取引先の状況に応じて、取引先と共に改善に取り組んでいます。

このCSR自主アセスメントは、WEBサーベイ形式のもので、取引先のCSR自主アセスメント実施状況を2024年4月に取引先に対して12,300社のうち95%が実施しました。

### 購入先様からのコミットメントの取得
当社グループは、「パナソニックグループ コンプライアンス方針」に基づき、サプライチェーン全体におけるCSRガイドラインの遵守を徹底しています。また、CSRガイドラインの遵守に関するコミットメントを取引先に求めています。新規取引先に対しては、取引開始前にCSRガイドラインの遵守を求めるとともに、既存取引先に対しても、定期的にコミットメントの取得を行っています。

## 環境負荷低減の取り組み
当社グループは環境負荷を低減し、物質・エネルギーの使用量を減少させる取り組みを行っています。

[サプライチェーン推進](https://holdings.panasonic/jp/corporate/sustainability/pdf/sdb2024-e.pdf)

## 責任ある調達活動
- 基本的な考え方
  - サプライチェーンにおける責任ある調達活動は、リスクの回避、持続可能な社会の実現に寄与することを目的としています。特に、環境、サプライヤー、労働、ハラスメントなどの問題に対して取り組んでいます。
```



  1%|          | 12/1539 [12:26<13:30:20, 31.84s/it]

./input/pdf_text/10_109.md
# 責任ある調達活動

## 方針
責任者・体制  
サプライチェーンのデュー・ディリジェンス  
環境負荷低減の取り組み  
責任ある調達活動  
社内教育と社外での啓発活動  
通報・相談窓口  

## 責任ある調達活動
PHDの調達活動は、環境保護、製品/サービスの品質向上、社会的責任を重視しており、サプライチェーン全体において持続可能性に配慮しています。

## デュー・ディリジェンスの取り組み
責任ある調達活動は、製品/サービスの調達においてのリスクを評価し、当社グループでは、調達ルールとして、RMIの実行する鉱・タングステン・金（3TG）の調達基準（CMRT）およびコバルト・ナイの調達基準（EMRT）を含む基準の遵守を徹底しています。

## 責任ある鉱物調達
責任グループでは、当社が直接または間接に購入した鉱物について、環境および社会的影響を考慮し、2023年度には、責任ある鉱物調達の対象鉱物が3,041品目から5,000品目以上に拡大され、EMRT基準に基づく調達が進められています。2024年には、より多くの鉱物に関しても調達基準を拡大することを検討しています。

2023年度、当社は、鉱物調達においてConformant / Active Smelter (RMIの認証を受けている、無鉱物についての調達基準を遵守している)を含む、責任ある調達の実施を進めています。サプライチェーンにおける重要な要素として、サプライヤーとの関係を強化し、持続可能な調達を推進しています。

## 業界連携の取り組み
当社グループでは、電子業界連盟（JEITA）「責任ある鉱物調達ガイドライン」に基づき、サプライチェーン全体における透明性を高めるために、国内外の関連団体と連携し、情報共有を行っています。また、JEITAのサプライヤーグループにも参加し、他の企業と協力しています。

## 社内教育と社外での啓発活動
責任調達では、調達活動が社会的責任を果たすために人材を育成することが重要です。CSRに関する考え方を浸透させるためにコンプライアンス教育を実施しています。また、各国（欧州/米国・中東・アジア）において、責任ある調達活動について、理解を深めるための研修を実施しています。


  1%|          | 12/1539 [12:35<26:42:56, 62.98s/it]


KeyboardInterrupt: 

In [ ]:
# # multiprocessing_ver
# ### 動作しないためNG

# def abstract_text_and_save(img_path):
#     md_text = get_text_from_img(img_path)
#     file_name = get_filename_without_extension(img_path)
#     with open(pdf_text_folder_path + f'{file_name}.md', 'w') as f:
#         f.write(md_text)

# if __name__ == '__main__':
#     # 画像パスのリストを取得

#     # プロセスプールを作成し、並列処理
#     with Pool(cpu_count()) as p:
#         list(tqdm(p.imap(abstract_text_and_save, 
#                          img_paths), 
#                   total=len(img_paths)))


  0%|          | 0/1539 [00:00<?, ?it/s]